In [63]:
import os
import pandas as pd
import math
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

# read in images and legend
imagesDir = "/root/sharedfolder/images"
images = os.listdir(imagesDir)
legend = pd.read_csv("/root/sharedfolder/data/legend.csv")

# convert all emotions classifications to lower-case
legend.emotion = legend['emotion'].str.lower()

# get count of each emotion
anger = legend.emotion.value_counts()['anger']
surprise = legend.emotion.value_counts()['surprise']
disgust = legend.emotion.value_counts()['disgust']
fear = legend.emotion.value_counts()['fear']
neutral = legend.emotion.value_counts()['neutral']
happiness = legend.emotion.value_counts()['happiness']
sadness = legend.emotion.value_counts()['sadness']
contempt = legend.emotion.value_counts()['contempt']

legendCnt = pd.DataFrame({
        'emotion': legend.emotion.unique(),
        'emotionCnt': (anger, surprise, disgust, fear,
                       neutral, happiness, sadness, contempt) })

legendCnt

,emotion,emotionCnt
0,anger,252
1,surprise,368
2,disgust,208
3,fear,21
4,neutral,6868
5,happiness,5696
6,sadness,268
7,contempt,9


In [64]:
# increase anger, surprise, disgust, fear, sadness, contempt 
# to 6000 each using transformations
legendCnt['trans'] = 0

legendCnt.trans[0] = math.ceil(6000/float(legendCnt.emotionCnt[0]))
legendCnt.trans[1] = math.ceil(6000/float(legendCnt.emotionCnt[1]))
legendCnt.trans[2] = math.ceil(6000/float(legendCnt.emotionCnt[2]))
legendCnt.trans[3] = math.ceil(6000/float(legendCnt.emotionCnt[3]))
legendCnt.trans[6] = math.ceil(6000/float(legendCnt.emotionCnt[6]))
legendCnt.trans[7] = math.ceil(6000/float(legendCnt.emotionCnt[7]))

legendCnt['transCnt'] = 0
legendCnt.transCnt[0] = legendCnt.emotionCnt[0] * legendCnt.trans[0]
legendCnt.transCnt[1] = legendCnt.emotionCnt[1] * legendCnt.trans[1]
legendCnt.transCnt[2] = legendCnt.emotionCnt[2] * legendCnt.trans[2]
legendCnt.transCnt[3] = legendCnt.emotionCnt[3] * legendCnt.trans[3]
legendCnt.transCnt[4] = legendCnt.emotionCnt[4]
legendCnt.transCnt[5] = legendCnt.emotionCnt[5]
legendCnt.transCnt[6] = legendCnt.emotionCnt[6] * legendCnt.trans[6]
legendCnt.transCnt[7] = legendCnt.emotionCnt[7] * legendCnt.trans[7]

legendCnt

,emotion,emotionCnt,trans,transCnt
0,anger,252,24,6048
1,surprise,368,17,6256
2,disgust,208,29,6032
3,fear,21,286,6006
4,neutral,6868,0,6868
5,happiness,5696,0,5696
6,sadness,268,23,6164
7,contempt,9,667,6003


In [65]:
legendCnt.transCnt.sum(axis=1)

49073

In [80]:
# augment anger, surprise, disgust, fear, sadness, contempt 
def picAugment(inDir, emoImg, augSize):
           
    # Using keras ImageDataGenerator to generate random images
    datagen = ImageDataGenerator(
        rotation_range = 40,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        vertical_flip = True)    
          
    for i in range(0,len(emoImg)):
        img = load_img(os.path.join(inDir, emoImg.iloc[i,1]))
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)
        j = 0
        for batch in datagen.flow_from_directory(x, batch_size = 1, 
                                                 save_to_dir = "/root/sharedfolder/images-balance"):
            j += 1
            if j >= augsize:
                break

In [67]:
angerImg = legend.query('emotion == "anger"')

In [68]:
surpriseImg = legend.query('emotion == "surprise"')

In [69]:
disgustImg = legend.query('emotion == "disgust"')

In [70]:
fearImg = legend.query('emotion == "fear"')

In [71]:
sadnessImg = legend.query('emotion == "sadness"')

In [72]:
contemptImg = legend.query('emotion == "contempt"')

In [73]:
angerImg[0:3]

,user.id,image,emotion
0,628,facial-expressions_2868588k.jpg,anger
223,302,Alejandro_Toledo_0004.jpg,anger
273,302,Alexander_Lukashenko_0001.jpg,anger


In [74]:
angerImg.iloc[0,2]

'anger'

In [81]:
picAugment(imagesDir, angerImg, 1)

TypeError: flow() takes at least 3 arguments (4 given)